In [21]:
import csv
import operator
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SpanishStemmer
import string
import json
from gensim.models import Phrases, LdaModel, LsiModel, TfidfModel
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk.stem as stem
porter = stem.PorterStemmer()

In [22]:
rows = []
with open('data.csv', newline='') as csvfile:
    datareader = csv.reader(csvfile)
    next(datareader)
    for row in datareader:
        rows.append(row)

In [23]:
rows[0][12]

'CLAVE: difusión de datos para desmitificar la discusión. También creo que hubo una buena sinergia entre las acciones en la calle y su posterior repercusión en redes'

In [24]:
rows[0][13]

'Entran en discusiones intramovimiento y la repetición de trabajo por falta de coordinación '

In [118]:
stop_words = stopwords.words('spanish') + list(string.punctuation) + \
['``', "''", '”', '“', 'mas', 'del','si', 'creo', 'aborto', 'sé', 'quizás', 'parece', 'sino', 'veces', 
 'siempre', 'aunque', 'solo', 'negativo', 'ninguna', 'acciones', 'personas', 'parte', 'todas', 'gente', 'falta', 
 'grupos', 'favor', 'muchas', 'puede', 'cierto', 'espacio', 'cantidad', 'muchas', 'estado', 'persona', 'masiva', 'negativa', 
 'negativas', 'vez', 'ser', 'mucha', 'tal', 'q', '...', 'cualquier', 'bien', 'decir', 'haber', 'tipo', 'tema'
]

In [119]:
replacements = {
    'iglesias': 'iglesia',
    'pañuelos': 'pañuelo', 
    'pañuelazos': 'pañuelazo',
    'movilizacion': 'movilización',
    'movilizaciones': 'movilización',
    'movimiento': 'movilización',
    'marcha': 'movilización',
    'marchas': 'movilización',
    'manifestaciones': 'movilización',
    'masivos': 'masivo',
    'masivas': 'masivo',
    'masiva': 'masivo',
    'debates': 'debate',
    'tv': 'medios',
    'television': 'medios',
    'televisión': 'medios',
    'calle': 'calles',
    'negativa': 'negativo',
    'negativas': 'negativo',
    'violentas': 'violencia',
    'violenta': 'violencia',
    'redes': 'redes_sociales'
}

In [120]:
phrases = {
    'pro vida': 'provida',
    'redes sociales': 'redes_sociales',
    'dela': 'de la'
}

def replace_phrases(text):
    for k, v in phrases.items():
        text = text.replace(k, v)
    return text

In [121]:
# stemmer = SpanishStemmer()
# stemmed_dictionary = {}
# def tokenize(text):
#     tokens = word_tokenize(replace_phrases(text))
#     return tokens
#     filtered_tokens = []
#     for token in tokens:
#         lower_token = token.lower()
        
#         if lower_token not in stop_words and len(lower_token) > 4:
#             lower_token = replacements.get(lower_token, lower_token)
#             stemmed_token = stemmer.stem(lower_token)
#             if stemmed_token in stemmed_dictionary:
#                 if lower_token in stemmed_dictionary[stemmed_token]:
#                     count_yet = stemmed_dictionary[stemmed_token][lower_token]
#                     stemmed_dictionary[stemmed_token][lower_token] = count_yet + 1
#                 else:
#                     stemmed_dictionary[stemmed_token][lower_token] = 1
#             else:
#                 stemmed_dictionary[stemmed_token] = {lower_token: 1}            
            
#             filtered_tokens.append(stemmed_token)
#         #else:
#             #print('ignore', lower_token)
#     return filtered_tokens

In [122]:
# all_positive_tokens = []
# all_negative_tokens = []
# positive_tokens = []
# negative_tokens = []
# for row in rows:
#     p_tokens = tokenize(row[12])
#     all_positive_tokens.extend(p_tokens)
#     positive_tokens.append(p_tokens)
#     n_tokens = tokenize(row[13])
#     all_negative_tokens.extend(n_tokens)
#     negative_tokens.append(n_tokens)

positive_tokens = []
negative_tokens = []
for row in rows:
    positive_tokens.extend(word_tokenize(row[12]))
    negative_tokens.extend(word_tokenize(row[13]))
    
positive = []
for w in positive_tokens:
    token = w.lower()
    token = replacements.get(token, token)
    if token not in stop_words:
        positive.append(token)
        
negative = []
for w in negative_tokens:
    token = w.lower()
    token = replacements.get(token, token)
    if token not in stop_words:
        negative.append(token)
        
        
#filtered_positive_tokens = [replacements.get(w.lower(), w.lower()) for w in positive_tokens if w.lower() not in stop_words]
#filtered_negative_tokens = [replacements.get(w.lower(), w.lower()) for w in negative_tokens if w.lower() not in stop_words]

In [123]:
positive_text = nltk.Text(positive)
negative_text = nltk.Text(negative)

In [124]:
p_dist = nltk.FreqDist(positive_text)
n_dist = nltk.FreqDist(negative_text)

In [125]:
p_dist.most_common(50)

[('movilización', 341),
 ('debate', 249),
 ('pañuelazo', 238),
 ('medios', 166),
 ('calles', 147),
 ('congreso', 145),
 ('redes_sociales', 120),
 ('mujeres', 115),
 ('pañuelo', 95),
 ('exposiciones', 93),
 ('masivo', 85),
 ('información', 82),
 ('sociales', 76),
 ('difusión', 58),
 ('actrices', 44),
 ('presencia', 40),
 ('comunicación', 38),
 ('verde', 37),
 ('apoyo', 35),
 ('datos', 34),
 ('testimonios', 34),
 ('vigilias', 32),
 ('intervenciones', 31),
 ('charlas', 29),
 ('campaña', 27),
 ('diferentes', 27),
 ('verdes', 27),
 ('feministas', 24),
 ('gran', 24),
 ('lucha', 24),
 ('hablar', 22),
 ('informacion', 21),
 ('participación', 21),
 ('figuras', 20),
 ('públicas', 20),
 ('visibilizar', 20),
 ('jóvenes', 20),
 ('argumentos', 20),
 ('sociedad', 20),
 ('visibilizacion', 20),
 ('espacios', 19),
 ('programas', 19),
 ('diputados', 19),
 ('pública', 19),
 ('difusion', 19),
 ('exposición', 19),
 ('visibilización', 18),
 ('distintos', 18),
 ('masividad', 18),
 ('abortos', 18)]

In [126]:
n_dist.most_common(50)

[('medios', 83),
 ('debate', 81),
 ('iglesia', 63),
 ('violencia', 63),
 ('movilización', 52),
 ('información', 41),
 ('antiderechos', 32),
 ('desinformación', 32),
 ('vida', 30),
 ('argumentos', 27),
 ('redes_sociales', 26),
 ('pro', 26),
 ('mujeres', 25),
 ('sectores', 23),
 ('pintadas', 23),
 ('legalización', 23),
 ('provida', 23),
 ('ley', 22),
 ('sociales', 20),
 ('lucha', 20),
 ('derechos', 20),
 ('comunicación', 18),
 ('legal', 18),
 ('campaña', 18),
 ('sociedad', 16),
 ('anti', 16),
 ('acción', 16),
 ('falsa', 14),
 ('senadores', 14),
 ('feministas', 14),
 ('opinión', 13),
 ('paredes', 13),
 ('hablar', 13),
 ('poder', 13),
 ('masivo', 13),
 ('pañuelo', 13),
 ('mal', 13),
 ('alguna', 12),
 ('discusión', 12),
 ('considero', 12),
 ('causa', 12),
 ('lado', 12),
 ('ejemplo', 11),
 ('cosas', 11),
 ('espacios', 11),
 ('etc', 11),
 ('agresiones', 11),
 ('postura', 11),
 ('hecho', 11),
 ('influencia', 11)]

In [127]:

features = []
for f in p_dist.most_common(50):
    features.append({
            'text': 'p_' + f[0],
            'size': f[1],
            'type': 'positive'
        })
for f in n_dist.most_common(50):
    features.append({
            'text': 'n_' + f[0],
            'size': f[1],
            'type': 'negative'
        })    
with open('most_informative_features.json', 'w') as f:
    json.dump(features, f)

In [252]:
features = []
for f in p_dist.most_common(50):
    features.append({
            'text': 'p_' + max(stemmed_dictionary[f[0]].items(), key=operator.itemgetter(1))[0],
            'size': f[1],
            'type': 'positive'
        })
for f in n_dist.most_common(50):
    features.append({
            'text': 'n_' + max(stemmed_dictionary[f[0]].items(), key=operator.itemgetter(1))[0],
            'size': f[1],
            'type': 'negative'
        })    
with open('most_informative_features.json', 'w') as f:
    json.dump(features, f)

//anaconda/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-12-26 11:40:37.897110. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [229]:
dictionary = corpora.Dictionary(positive_tokens)
dictionary.filter_extremes(0.05, 10)
dictionary.compactify()
corpus = [dictionary.doc2bow(token) for token in tokens]
lda = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

//anaconda/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-12-26 11:21:13.866410. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [222]:
topics = lda.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.086*"gent" + 0.027*"darl" + 0.017*"social" + 0.016*"mujer"')
(1, '0.093*"accion" + 0.056*"favor" + 0.041*"mujer" + 0.019*"part"')
(2, '0.040*"senador" + 0.019*"sexual" + 0.018*"mayor" + 0.016*"discusion"')
(3, '0.055*"sector" + 0.032*"part" + 0.022*"posicion" + 0.016*"comunicacion"')
(4, '0.120*"debat" + 0.029*"parec" + 0.022*"opinion" + 0.015*"call"')


In [223]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)

print("LSA Topics:")
for i in lsi.show_topics():
    print("Topic: ", i[0], i[1])

LSA Topics:
Topic:  0 -0.902*"debat" + -0.374*"accion" + -0.161*"gent" + -0.065*"favor" + -0.062*"sector" + -0.049*"mujer" + -0.041*"parec" + -0.039*"pued" + -0.034*"senador" + -0.031*"opinion"
Topic:  1 -0.901*"accion" + 0.389*"debat" + -0.136*"sector" + -0.067*"mujer" + -0.039*"favor" + 0.039*"gent" + -0.036*"grand" + -0.032*"mejor" + -0.031*"mayor" + -0.030*"part"
Topic:  2 -0.973*"gent" + 0.171*"debat" + -0.088*"favor" + -0.061*"parec" + -0.051*"opinion" + 0.040*"accion" + -0.036*"part" + -0.031*"mujer" + -0.027*"sexual" + -0.027*"suficient"
Topic:  3 0.921*"favor" + 0.255*"sector" + -0.112*"accion" + -0.107*"gent" + 0.106*"legal" + 0.098*"part" + 0.068*"mujer" + 0.059*"opinion" + 0.059*"posicion" + 0.057*"personaj"
Topic:  4 -0.930*"sector" + 0.251*"favor" + 0.133*"mujer" + -0.133*"informacion" + 0.113*"accion" + 0.071*"part" + -0.043*"respond" + -0.042*"expresion" + -0.038*"gent" + 0.032*"legal"


In [224]:
dictionary = corpora.Dictionary(negative_tokens)
dictionary.filter_extremes(0.05, 10)
dictionary.compactify()
corpus = [dictionary.doc2bow(token) for token in tokens]
lda = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
topics = lda.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.040*"gent" + 0.040*"favor" + 0.032*"part" + 0.021*"debat"')
(1, '0.041*"quien" + 0.028*"ley" + 0.027*"senador" + 0.019*"agresion"')
(2, '0.046*"debat" + 0.021*"legal" + 0.019*"opinion" + 0.015*"lugar"')
(3, '0.050*"sector" + 0.021*"informacion" + 0.016*"ataqu" + 0.012*"posicion"')
(4, '0.082*"accion" + 0.036*"mujer" + 0.020*"poder" + 0.020*"mal"')


In [225]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)

print("LSA Topics:")
for i in lsi.show_topics():
    print("Topic: ", i[0], i[1])

LSA Topics:
Topic:  0 -0.888*"debat" + -0.407*"accion" + -0.106*"gent" + -0.071*"favor" + -0.065*"sector" + -0.058*"mujer" + -0.053*"quien" + -0.047*"pued" + -0.044*"pared" + -0.036*"mal"
Topic:  1 0.873*"accion" + -0.432*"debat" + 0.130*"sector" + 0.101*"quien" + 0.067*"mujer" + 0.045*"ley" + 0.039*"part" + 0.036*"favor" + 0.036*"grand" + 0.033*"mayor"
Topic:  2 0.947*"gent" + 0.171*"favor" + -0.119*"debat" + 0.084*"porqu" + 0.077*"part" + -0.075*"accion" + 0.075*"opinion" + 0.072*"quien" + 0.067*"ley" + 0.064*"bien"
Topic:  3 0.601*"favor" + 0.521*"quien" + 0.298*"ley" + -0.239*"gent" + 0.231*"part" + 0.182*"sector" + -0.175*"accion" + 0.153*"mal" + 0.100*"parec" + 0.074*"bien"
Topic:  4 -0.663*"sector" + 0.499*"quien" + -0.381*"favor" + 0.262*"part" + -0.176*"informacion" + 0.070*"posibl" + 0.067*"mujer" + 0.063*"pared" + 0.061*"parec" + 0.058*"ley"


In [227]:
import pyLDAvis.gensim

In [230]:
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

//anaconda/envs/py36/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-12-26 11:21:18.808181. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
